In [ ]:
# summary of how to connect to db and populate with columns in a csv
import csv
import psycopg2
conn = psycopg2.connect(user="ashkanahmadidastgerdi", password="", host="localhost",port="6543",database="ashkanahmadidastgerdi")
cur = conn.cursor()


with open('iv_swinger.csv', "r") as csv_file:
    csv_reader = csv.DictReader(csv_file, delimiter=',')
    for lines in csv_reader:
        cur.execute(
        "INSERT INTO production.panel_iv_readings (panel_id, created_At, voltage, current) VALUES (2, now(), %s, %s)",
        (lines['Volts'], lines[' Amps'])
    )
conn.commit()

In [ ]:
#summary on how to get latest file path in a directory
import glob
import os

while True:
    list_of_files = glob.glob('/Users/ashkanahmadidastgerdi/mydev/vistella/core/*') # * means all if need specific format then *.csv
    latest_file = max(list_of_files, key=os.path.getctime)
    #print(latest_file)

In [ ]:
import time
import os
import csv
import psycopg2
from watchdog.observers import Observer
from watchdog.events import PatternMatchingEventHandler
import tkinter as tk

if __name__ == "__main__":
    patterns = "*"
    ignore_patterns = ""
    ignore_directories = False
    case_sensitive = True
    my_event_handler = PatternMatchingEventHandler(patterns, ignore_patterns, ignore_directories, case_sensitive)


def clicked():
    conn = psycopg2.connect(user="ashkanahmadidastgerdi", password="", host="localhost",port="6543",database="ashkanahmadidastgerdi")
    cur = conn.cursor()
    filepath = file.cget("text")
    with open(filepath, "r") as csv_file:
        csv_reader = csv.DictReader(csv_file, delimiter=',')
        for lines in csv_reader:
            cur.execute("INSERT INTO production.panel_iv_readings (panel_id, created_At, voltage, current) VALUES (%s, now(), %s, %s)", (txt.get(), lines['Volts'], lines[' Amps']))
    conn.commit()
    txt.delete(0, 'end')
    successMessage.config(text = "Succesfully submitted and Waiting")
    btn_save["state"] = "disabled"
    btn_del["state"] = "disabled"
    window.update()

def clicked_cancel():
        txt.delete(0, 'end')
        successMessage = tk.Label(window,  text = "Waiting")
        btn_save["state"] = "disabled"
        btn_del["state"] = "disabled"
        pass

def on_created(event):
    #print(f"hey, {event.src_path} has been created!")   
    a = os.path.basename(event.src_path)
    b = event.src_path
    print(a)
    if a[-3:] == "csv" and a[0:2] == "iv" :
        print("File found")
        #Ask user if the data should be saved and with which panel ID
        btn_save["state"] = "normal"
        btn_del["state"] = "normal"
        successMessage.config(text = "Data received. Choose action")
        file.config(text = b)
    else:
        pass
    
my_event_handler.on_created = on_created


path = "/Users/ashkanahmadidastgerdi/Library/Application Support/IV_Swinger2"
go_recursively = True
my_observer = Observer()
my_observer.schedule(my_event_handler, path, recursive=go_recursively)

my_observer.start()

#Define tkinter
window = tk.Tk()
window.title("Enter panel ID")
window.geometry('350x200')
lbl = tk.Label(window, text="Panel ID:")
lbl.grid(column=0, row=1)
txt = tk.Entry(window,width=10)
txt.grid(column=1, row=1)
btn_save = tk.Button(window, text="Save to DB", command=clicked)
btn_save.grid(column=2, row=1)
btn_save["state"] = "disabled"
btn_del = tk.Button(window, text="Cancel", command=clicked_cancel)
btn_del.grid(column=3, row=1)
btn_del["state"] = "disabled"
successMessage = tk.Label(window,  text = "Waiting")
successMessage.grid(column=0, row=3, columnspan = 3,)
file = tk.Label(window,  text = "")
successMessage.grid(column=0, row=4, columnspan = 2,)
window.mainloop()
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    my_observer.stop()
    my_observer.join()
    
